In [2]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

# Carregar os dados
df = pd.read_csv('processed-dataset.csv', parse_dates=['checkin'], index_col='checkin')

In [3]:
# Feature Engineering
df['dia_da_semana'] = df.index.dayofweek
df['mes'] = df.index.month
df['ano'] = df.index.year
df['lag_1'] = df['preço'].shift(1)
df['lag_2'] = df['preço'].shift(2)
df['rolling_mean_2'] = df['preço'].rolling(window=2).mean()
df['rolling_mean_4'] = df['preço'].rolling(window=4).mean()

df = df.dropna()

In [4]:
# Selecionar as features para o modelo
features = df[['dia_da_semana', 'mes', 'ano', 'lag_1', 'lag_2', 'rolling_mean_2', 'rolling_mean_4']]
target = df['preço']

# Configuração da validação cruzada
tscv = TimeSeriesSplit(n_splits=5)

mae_scores = []

# Iniciar um experimento no MLflow
mlflow.set_experiment("Previsão de Preços de Hotéis")

with mlflow.start_run():
    for train_index, test_index in tscv.split(features):
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]

        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        mae = mean_absolute_error(y_test, predictions)
        mae_scores.append(mae)

        print(f"Train Index: {train_index}, Test Index: {test_index}")
        print(f"MAE: {mae}")

        # Registrar parâmetros, métricas e artefatos no MLflow
        mlflow.log_param("n_estimators", 100)
        mlflow.log_metric("mae", mae)

    mean_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)

    mlflow.log_metric("mean_mae", mean_mae)
    mlflow.log_metric("std_mae", std_mae)
    
    # Registrar o modelo no MLflow
    mlflow.sklearn.log_model(model, "random_forest_model")

print(f"Mean MAE: {mean_mae}, Std MAE: {std_mae}")

Train Index: [   0    1    2 ... 2433 2434 2435], Test Index: [2436 2437 2438 ... 4865 4866 4867]
MAE: 48.54488898026315
Train Index: [   0    1    2 ... 4865 4866 4867], Test Index: [4868 4869 4870 ... 7297 7298 7299]
MAE: 20.79071957236842
Train Index: [   0    1    2 ... 7297 7298 7299], Test Index: [7300 7301 7302 ... 9729 9730 9731]
MAE: 14.677397203947368
Train Index: [   0    1    2 ... 9729 9730 9731], Test Index: [ 9732  9733  9734 ... 12161 12162 12163]
MAE: 12.03175575657895
Train Index: [    0     1     2 ... 12161 12162 12163], Test Index: [12164 12165 12166 ... 14593 14594 14595]
MAE: 16.253248355263157
Mean MAE: 22.459601973684208, Std MAE: 13.34952343514642


In [17]:

# Carregar o modelo treinado do MLflow
model_path = "mlruns/391920513749379523/7c197ca336754e5c9eab1d4e8d5a9c1c/artifacts/random_forest_model"
loaded_model = mlflow.sklearn.load_model(model_path)

# Dados de entrada para a predição (exemplo)
# Aqui, você precisa fornecer as mesmas features que foram usadas durante o treinamento do modelo
# Este é um exemplo fictício para ilustração
new_data = pd.DataFrame({
    'dia_da_semana': [1],       # Exemplo: Quarta-feira
    'mes': [3],                 # Exemplo: Julho
    'ano': [2025],              # Exemplo: 2024
    'lag_1': [250],             # Valor do preço do último período
    'lag_2': [240],             # Valor do preço do penúltimo período
    'rolling_mean_2': [245],    # Média móvel dos últimos 2 períodos
    'rolling_mean_4': [235],    # Média móvel dos últimos 4 períodos
})

# Realizar a predição
predicted_price = loaded_model.predict(new_data)

# Exibir o preço previsto
print(f"Preço previsto do hotel: {predicted_price[0]}")


Preço previsto do hotel: 519.3
